# Similarity Search - A Collection of Similarity Search Algorithms

Similarity search in machine learning refers to the task of finding objects that are similar to a given query object within a dataset. In this notebook, I'll be implementing multiple Similarity Search algorithms as describe in <a href="https://www.youtube.com/watch?v=AY62z7HrghY&list=PLIUOU7oqGTLhlWpTz4NnuT3FekouIVlqc&index=1" target="__blank__">this</a> YouTube playlist by James Briggs.

#### Jaccard Similarity

Jaccard similarity is a measure used to compare the similarity and diversity of sample sets. It is defined as the size of the intersection of the sets divided by the size of the union of the sets.

Mathematically, the Jaccard similarity J(A,B) between two sets A and B is calculated as:

$$
J(A, B) = | {A \cap B} | \div | {A \cup B} |
$$

The Jaccard similarity ranges from 0 to 1. A value of 1 indicates that the sets are identical, while a value of 0 indicates that the sets have no elements in common.

In [1]:
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

In [2]:
set1 = {1, 2, 3}
set2 = {1, 2, 3}

# 3 / 3
similarity = jaccard_similarity(set1, set2)
print("Jaccard Similarity:", similarity)

Jaccard Similarity: 1.0


In [3]:
set1 = {1, 2, 3, 4, 5}
set2 = {3, 4, 5, 6, 7}

# 3 / 7
similarity = jaccard_similarity(set1, set2)
print("Jaccard Similarity:", similarity)

Jaccard Similarity: 0.42857142857142855


In [4]:
set1 = {"A", "B", "C"}
set2 = {"A", "B", "D"}

# 2 / 4 
similarity = jaccard_similarity(set1, set2)
print("Jaccard Similarity:", similarity)

Jaccard Similarity: 0.5


#### W-shingling

W-shingling is a technique used in text analysis and information retrieval to represent documents as sets of shingles. Shingles, also known as n-grams, are contiguous sequences of words or characters in a document.

In w-shingling, a document is divided into w consecutive words (or characters) at a time, forming overlapping or non-overlapping shingles. The choice of w determines the size of the shingles.

For example, suppose we have the following sentence:

"Machine learning is a subset of artificial intelligence."

If we use 3-shingling (trigrams) with overlapping shingles, the shingles would be:

- "Machine learning is"
- "learning is a"
- "is a subset"
- "a subset of"
- "subset of artificial"
- "of artificial intelligence"

If we use non-overlapping shingles, the shingles would be:

- "Machine learning is"
- "a subset of"
- "artificial intelligence"

Here is an example of how the shingles can be generated,

In [5]:
def generate_shingles(text, w, overlap=False):
    shingles = set()
    words = text.split()
    
    if overlap:
        for i in range(len(words) - w + 1):
            shingle = " ".join(words[i:i+w])
            shingles.add(shingle)
    else:
        for i in range(0, len(words), w):
            shingle = " ".join(words[i:i+w])
            shingles.add(shingle)
    
    return shingles

text = "Machine learning is a subset of artificial intelligence."
w = 3  
overlap = True

shingles = generate_shingles(text, w, overlap)

print("Shingles:")
for shingle in shingles:
    print(shingle)

Shingles:
learning is a
Machine learning is
subset of artificial
is a subset
of artificial intelligence.
a subset of


Here is an example of how the generated shingles can be used with Jaccard similarity for similarity search,

In [6]:
text1 = "Machine learning is a subset of artificial intelligence"
text2 = "Artificial intelligence is a subset of computer science"

# Generate shingles for both texts
shingles_text1 = generate_shingles(text1, w, overlap)
shingles_text2 = generate_shingles(text2, w, overlap)

print(f"Shingles Text1: {shingles_text1}")
print(f"Shingles Text2: {shingles_text2}")

# Calculate Jaccard similarity between the sets of shingles
# Jaccard similarity = 2 / 10
similarity = jaccard_similarity(shingles_text1, shingles_text2)

# Print the Jaccard similarity
print("Jaccard Similarity:", similarity)

Shingles Text1: {'learning is a', 'Machine learning is', 'subset of artificial', 'is a subset', 'of artificial intelligence', 'a subset of'}
Shingles Text2: {'is a subset', 'intelligence is a', 'subset of computer', 'of computer science', 'Artificial intelligence is', 'a subset of'}
Jaccard Similarity: 0.2


#### Levenshtein distance

Levenshtein distance, also known as edit distance, is a measure of the similarity between two strings. It is defined as the minimum number of single-character edits (insertions, deletions, or substitutions) required to change one string into the other.

For example, the Levenshtein distance between the strings "kitten" and "sitting" is 3, as illustrated by the following sequence of edits:

- Replace 'k' with 's' (substitution)
- Insert 's' at the beginning
- Replace 'e' with 'i'


In [7]:
def levenshtein_distance(s1, s2):
    distance_matrix = [[0] * (len(s2) + 1) for _ in range(len(s1) + 1)]

    # Initialize the first row and column of the matrix
    # max(i,j) if min(i, j) = 0
    for i in range(len(s1) + 1):
        distance_matrix[i][0] = i
    for j in range(len(s2) + 1):
        distance_matrix[0][j] = j

    # Fill in the rest of the matrix
    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            if s1[i - 1] == s2[j - 1]:
                substitution_cost = 0
            else:
                substitution_cost = 1
            distance_matrix[i][j] = min(distance_matrix[i - 1][j] + 1, # deletion
                                        distance_matrix[i][j - 1] + 1, # insertion
                                        distance_matrix[i - 1][j - 1] + substitution_cost) # substitution

    # Return the bottom-right cell of the matrix
    return distance_matrix[len(s1)][len(s2)]

# Example usage:
s1 = "kitten"
s2 = "sitting"
distance = levenshtein_distance(s1, s2)
print("Levenshtein Distance between '{}' and '{}': {}".format(s1, s2, distance))


Levenshtein Distance between 'kitten' and 'sitting': 3


If you're looking for a fast implementation of Levenshtein Distance, check out this <a href="https://pypi.org/project/python-Levenshtein/" target="__blank__">Python module</a>.

#### TF-IDF

TF-IDF (Term Frequency-Inverse Document Frequency) is a method for weighting the importance of words in documents. It assigns higher weights to terms that are frequent in a document but rare across the entire corpus. This technique is commonly used in similarity search by representing documents as numerical vectors based on their TF-IDF scores. Cosine similarity is then applied to compare these vectors, determining the similarity between documents.

The steps for calculating TF-IDF are as follows:

1. Term Frequency (TF): Calculate the frequency of each term in a document relative to the total number of terms in that document.

2. Inverse Document Frequency (IDF): Calculate the logarithm of the ratio of the total number of documents to the number of documents containing each term.

3. TF-IDF Score: Multiply the TF and IDF values for each term in each document to get the TF-IDF score.

In [8]:
import math
from collections import Counter

def calculate_tf(term_freq):
    total_terms = sum(term_freq.values())
    tf_scores = {term: freq / total_terms for term, freq in term_freq.items()}
    return tf_scores

def calculate_idf(documents):
    total_docs = len(documents)
    all_terms = set(term for doc in documents for term in doc)
    idf_scores = {}

    for term in all_terms:
        doc_freq = sum(1 for doc in documents if term in doc)
        idf_scores[term] = math.log(total_docs / doc_freq) + 1
    return idf_scores

def calculate_tf_idf(tf_scores, idf_scores):
    tf_idf_scores = {}
    for term, tf_score in tf_scores.items():
        tf_idf_scores[term] = tf_score * idf_scores[term]
    return tf_idf_scores

# Sample documents
documents = [
    ["hello", "world", "hello"],
    ["hello", "python"],
    ["python", "world"]
]

# Calculate TF scores for each document
tf_scores_list = [calculate_tf(Counter(doc)) for doc in documents]

print(f"TF scores: {tf_scores_list}")

# Calculate IDF scores for the entire corpus
idf_scores = calculate_idf(documents)

print(f"IDF scores: {idf_scores}")

# Calculate TF-IDF scores for each document
tf_idf_scores_list = [calculate_tf_idf(tf_scores, idf_scores) for tf_scores in tf_scores_list]

# Print TF-IDF scores for each document
for i, tf_idf_scores in enumerate(tf_idf_scores_list):
    print(f"Document {i+1} TF-IDF scores:")
    for term, score in tf_idf_scores.items():
        print(f"{term}: {score:.4f}")
    print()


TF scores: [{'hello': 0.6666666666666666, 'world': 0.3333333333333333}, {'hello': 0.5, 'python': 0.5}, {'python': 0.5, 'world': 0.5}]
IDF scores: {'hello': 1.4054651081081644, 'world': 1.4054651081081644, 'python': 1.4054651081081644}
Document 1 TF-IDF scores:
hello: 0.9370
world: 0.4685

Document 2 TF-IDF scores:
hello: 0.7027
python: 0.7027

Document 3 TF-IDF scores:
python: 0.7027
world: 0.7027



#### Best Matching 25 (BM25)

BM25, or Best Matching 25, is a ranking function used to estimate the relevance of documents to a given search query. It is an improvement over the TF-IDF model and is widely used in information retrieval and search engines. BM25 takes into account factors such as term frequency, document length, and document frequency.

The score of a document $D$ given a query $Q$ which contains $q_1, q_2, ..., q_n$ is given by:


$$
\begin{equation} \text{BM25}(D,Q) = \sum_{i=1}^{n} \text{IDF}(q_i) \cdot \frac{f(q_i, D) \cdot (k_1 + 1)}{f(q_i, D) + k_1 \cdot (1 - b + b \cdot \frac{|D|}{\text{avgdl}})} \end{equation}
$$

where is $f(q_i, D)$'s term frequency in the document $D$, $|D|$ is the length of the document $D$ in words, and avgdl is the average document length in the text collection from which documents are drawn. $k_1$ and $b$ are free parameters.

In [9]:
import math
from collections import Counter

class BM25:
    def __init__(self, corpus, k1=1.5, b=0.75):
        self.corpus = corpus
        self.k1 = k1
        self.b = b
        self.avgdl = sum(map(len, corpus)) / len(corpus)
        self.idf = self.calculate_idf()
        
    def calculate_idf(self):
        idf = {}
        for document in self.corpus:
            word_set = set(document)
            for word in word_set:
                idf[word] = idf.get(word, 0) + 1
        for word, freq in idf.items():
            idf[word] = math.log((len(self.corpus) - freq + 0.5) / (freq + 0.5) + 1)
        return idf
    
    def get_score(self, query):
        scores = Counter()
        for word in query:
            if word in self.idf:
                for i, doc in enumerate(self.corpus):
                    f = doc.count(word)
                    dl = len(doc)
                    scores[i] += self.idf[word] * (f * (self.k1 + 1)) / (f + self.k1 * (1 - self.b + self.b * dl / self.avgdl))
        return scores


corpus = [['hello', 'world'], ['world', 'python'], ['python', 'example']]
bm25 = BM25(corpus)
query = ['hello']
scores = bm25.get_score(query)

# Scores for documents/corpus
print(scores)


Counter({0: 0.9808292530117264, 1: 0.0, 2: 0.0})


#### FAISS (Facebook AI Similarity Search)

FAISS, or Facebook AI Similarity Search, is an efficient library for similarity search and clustering of dense vectors. It is particularly popular for handling large-scale vector databases. FAISS is built to perform similarity searches using methods like k-nearest neighbors (KNN) and approximate nearest neighbors (ANN) efficiently. Here are the basic steps to use FAISS:

- Prepare data: Prepare your dataset in a suitable format (usually a numpy array).
- Choose index type: Select an appropriate index type based on your needs (e.g., Flat, IVF, LSH).
- Build index: Build the index using the chosen index type and the prepared data.
- Query: Perform queries using the index to find similar vectors or data points.
- Evaluate results: Evaluate and analyze the results obtained from the queries.
- Fine-tune parameters: Optionally fine-tune parameters to optimize performance for your specific use case.

In [10]:
# Install faiss
# !pip install faiss-cpu

In [11]:
import numpy as np
import faiss

np.random.seed(42)

d = 500
n_input = 100000
data = np.random.rand(n_input, d).astype(np.float32)

index = faiss.IndexFlatL2(d)
print(index.ntotal) # 0 because we have not added any data in yet

0


In [12]:
# Add data vectors to the index
index.add(data)

print(index.ntotal) # Same as n_input

100000


Now, let's query our index,

In [13]:
%%time

# Define a query vector
query = np.random.rand(1, d).astype(np.float32)

# Perform a nearest neighbor search
k = 3  # Number of nearest neighbors to retrieve
distances, indices = index.search(query, k)

print("Nearest neighbors:")
for i in range(k):
    print("Index:", indices[0][i], "Distance:", distances[0][i])
    # print("Vector:", data[indices[0][i]])

Nearest neighbors:
Index: 65415 Distance: 66.50713
Index: 41447 Distance: 67.38054
Index: 46569 Distance: 68.09364
CPU times: total: 0 ns
Wall time: 28 ms


As we can see the computation time to query is large given our index is large. We can improve speed of our query using a quantizer,

In [14]:
nlist = 1000
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

# Train using the embeddings
index.train(data)

print(f"Is trained? {index.is_trained}")
print(f"{index.ntotal}")

Is trained? True
0


In [15]:
# Adding to the index
index.add(data)
print(f"{index.ntotal}")

100000


In [16]:
%%time
k = 3 
distances, indices = index.search(query, k)

print("Nearest neighbors:")
for i in range(k):
    print("Index:", indices[0][i], "Distance:", distances[0][i])
    # print("Vector:", data[indices[0][i]])

Nearest neighbors:
Index: 41447 Distance: 67.38054
Index: 23145 Distance: 70.838776
Index: 20121 Distance: 70.900505
CPU times: total: 0 ns
Wall time: 1 ms


The computation time has gone down by a lot but we do suffer in accuracy by quantizing our data by a lot (splitting into 1000 buckets)! It is important to choose a value for `n_list` that is appropriate.

Using an argument called `nprobe` to be more exhaustive during our search by looking into nearby buckets.

In [17]:
%%time
index.nprobe = 400

k = 3 
distances, indices = index.search(query, k)

print("Nearest neighbors:")
for i in range(k):
    print("Index:", indices[0][i], "Distance:", distances[0][i])
    # print("Vector:", data[indices[0][i]])


Nearest neighbors:
Index: 65415 Distance: 66.50713
Index: 41447 Distance: 67.38054
Index: 46569 Distance: 68.09364
CPU times: total: 0 ns
Wall time: 14 ms


It's a bit more accurate now although our computation time has gone up.

Next, FAISS also provides a way to quantize the data using product quantization.

In [18]:
# Number of centroids in the constructed vector ( d % m should be 0 )
m = 5

# Number of bits insde centroid
bits = 8

quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits)

In [19]:
index.is_trained

False

In [20]:
# Train on the embedding data
index.train(data)

In [21]:
index.add(data)

In [22]:
%%time
distances, indices = index.search(query, k)

print("Nearest neighbors:")
for i in range(k):
    print("Index:", indices[0][i], "Distance:", distances[0][i])
    # print("Vector:", data[indices[0][i]])

Nearest neighbors:
Index: 45632 Distance: 42.138412
Index: 14152 Distance: 42.256477
Index: 84948 Distance: 42.453526
CPU times: total: 0 ns
Wall time: 998 µs


Its a lot faster but the accuracy has suffered compared to a fully exhaustive search.

Now, let's move onto other kind of indexes that are available in FAISS,

- IndexFlatL2 - A type of FAISS index that creates a flat index structure using L2 (Euclidean distance) metric for similarity search. It directly stores all vectors in a single structure without any hierarchical organization, making it simple but potentially less efficient for large datasets.

In [23]:
%%time

index = faiss.IndexFlatL2(d)
index.add(data)

k=10
distances, indices = index.search(query, k)

CPU times: total: 141 ms
Wall time: 73.5 ms


Creating a baseline for future comparisons,

In [24]:
baseline = indices[0].tolist()
baseline

[65415, 41447, 46569, 99294, 46063, 76476, 73468, 11862, 24210, 85898]

- IndexFlatIP - FAISS index optimized for inner product similarity search, employing a flat structure to efficiently compute inner products between query vectors and dataset vectors, commonly used in recommendation systems and information retrieval.

In [25]:
index = faiss.IndexFlatIP(d)
index.add(data)

In [26]:
%%time
distances, indices = index.search(query, k)

CPU times: total: 0 ns
Wall time: 14 ms


In [27]:
np.in1d(baseline, indices)

array([False, False, False, False, False, False, False, False, False,
       False])

- Locality Sensitive Hashing (LSH) - LSH is a method for approximate nearest neighbor search. It hashes data points into hash buckets in a way that similar points are likely to hash to the same bucket. This enables efficient retrieval of approximate nearest neighbors by searching within these hash buckets rather than comparing all data points directly. LSH is especially useful for high-dimensional data where traditional exact nearest neighbor search methods become computationally expensive.

In [28]:
nbits = d * 2
index = faiss.IndexLSH(d, nbits)
index.add(data)

In [29]:
%%time
distances, indices = index.search(query, k)

CPU times: total: 0 ns
Wall time: 4.01 ms


In [30]:
np.in1d(baseline, indices)

array([False, False, False, False, False, False, False, False, False,
       False])

- Hierarchical Navigable Small World (HNSW) - A type of FAISS index that constructs a hierarchical graph structure to organize data points. It uses navigable small-world graphs to efficiently perform approximate nearest neighbor searches, especially effective for high-dimensional data.

In [31]:
M = 16 # number of connections each vertex as
ef_search = 8 # depth of our search
ef_construction = 64 # how efficiently and accurately are we going to build the graph

In [32]:
index = faiss.IndexHNSWFlat(d, M)
index.hnsw.efSearch = ef_search
index.hnsw.efConstruction = ef_construction

index.add(data)

In [33]:
%%time
distances, indices = index.search(query, k)
np.in1d(baseline, indices)


CPU times: total: 0 ns
Wall time: 0 ns


array([False,  True, False, False, False, False, False, False, False,
       False])